In [12]:
import numpy as np
import matplotlib.pyplot as plt
import random
import time
import math

In [13]:
# ------------------------ COMMON SETUP ------------------------
np.random.seed(42)
num_cities = 20
cities = np.random.rand(num_cities, 2)

def total_distance(path):
    return sum(np.linalg.norm(cities[path[i]] - cities[path[(i + 1) % len(path)]]) for i in range(len(path)))

def get_neighbor(path):
    a, b = random.sample(range(len(path)), 2)
    new_path = path.copy()
    new_path[a], new_path[b] = new_path[b], new_path[a]
    return new_path

In [14]:
# ------------------------ HILL CLIMBING ------------------------
def run_hill_climbing():
    path = list(range(num_cities))
    random.shuffle(path)
    best_path = path.copy()
    best_distance = total_distance(best_path)

    for _ in range(1000):
        neighbor = get_neighbor(best_path)
        dist = total_distance(neighbor)
        if dist < best_distance:
            best_path = neighbor
            best_distance = dist
    return best_distance, best_path

In [15]:
# ------------------------ SIMULATED ANNEALING ------------------------
def run_simulated_annealing():
    initial_temp = 100
    cooling_rate = 0.995
    min_temp = 1e-3

    path = list(range(num_cities))
    random.shuffle(path)
    current_path = path.copy()
    current_distance = total_distance(current_path)
    best_path = path.copy()
    best_distance = current_distance

    temperature = initial_temp
    while temperature > min_temp:
        neighbor = get_neighbor(current_path)
        neighbor_distance = total_distance(neighbor)
        delta = neighbor_distance - current_distance

        if delta < 0 or random.random() < math.exp(-delta / temperature):
            current_path = neighbor
            current_distance = neighbor_distance

            if current_distance < best_distance:
                best_path = current_path
                best_distance = current_distance

        temperature *= cooling_rate

    return best_distance, best_path

In [16]:
# ------------------------ RUN MULTIPLE TIMES ------------------------
hc_distances, hc_times, hc_paths = [], [], []
sa_distances, sa_times, sa_paths = [], [], []

for i in range(5):
    start = time.time()
    d, p = run_hill_climbing()
    t = time.time() - start
    hc_distances.append(d)
    hc_times.append(t)
    hc_paths.append(p)

    start = time.time()
    d, p = run_simulated_annealing()
    t = time.time() - start
    sa_distances.append(d)
    sa_times.append(t)
    sa_paths.append(p)

In [17]:
# ------------------------ STATS + PLOTTING ------------------------

avg_hc_time = np.mean(hc_times)
avg_sa_time = np.mean(sa_times)
avg_hc_dist = np.mean(hc_distances)
avg_sa_dist = np.mean(sa_distances)

print(f"Avg HC Distance: {avg_hc_dist:.3f}, Avg Time: {avg_hc_time:.4f}s")
print(f"Avg SA Distance: {avg_sa_dist:.3f}, Avg Time: {avg_sa_time:.4f}s")

labels = ['Hill Climbing', 'Simulated Annealing']

# Runtime and Distance Comparison
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.bar(labels, [avg_hc_time, avg_sa_time], color=['blue', 'green'])
plt.ylabel("Avg Runtime (s)")
plt.title("Avg Runtime Comparison")

plt.subplot(1, 2, 2)
plt.bar(labels, [avg_hc_dist, avg_sa_dist], color=['blue', 'green'])
plt.ylabel("Avg Final Distance")
plt.title("Avg Path Distance Comparison")

# Save charts as PNG
plt.tight_layout()
plt.savefig("runtime_distance_comparison.png")
plt.close()

Avg HC Distance: 4.717, Avg Time: 0.0434s
Avg SA Distance: 4.574, Avg Time: 0.0998s


In [18]:
# ------------------------ LINE CHART FOR EACH RUN ------------------------

# Plot Line Charts for Time Comparison
plt.figure(figsize=(10, 5))
plt.plot(range(1, 6), hc_times, label="Hill Climbing", marker='o', color='blue')
plt.plot(range(1, 6), sa_times, label="Simulated Annealing", marker='o', color='green')
plt.xlabel("Run Number")
plt.ylabel("Runtime (seconds)")
plt.title("Runtime Comparison for Each Run")
plt.legend()

# Save line chart for runtime
plt.savefig("runtime_comparison_line_chart.png")
plt.close()

# Plot Line Charts for Distance Comparison
plt.figure(figsize=(10, 5))
plt.plot(range(1, 6), hc_distances, label="Hill Climbing", marker='o', color='blue')
plt.plot(range(1, 6), sa_distances, label="Simulated Annealing", marker='o', color='green')
plt.xlabel("Run Number")
plt.ylabel("Final Distance")
plt.title("Final Distance Comparison for Each Run")
plt.legend()

# Save line chart for distance
plt.savefig("distance_comparison_line_chart.png")
plt.close()

In [19]:
# ------------------------ BEST ROUTE PLOTS ------------------------

def plot_best_route(best_path, algo_name, color, filename):
    coords = cities[best_path + [best_path[0]]]  # loop back to start
    plt.figure(figsize=(6, 6))
    plt.plot(coords[:, 0], coords[:, 1], marker='o', color=color, lw=2)
    plt.scatter(cities[:, 0], cities[:, 1], c='black')
    plt.scatter(*cities[best_path[0]], c='green', s=100, label='Start')
    plt.scatter(*cities[best_path[-1]], c='red', s=100, label='End')
    plt.title(f"Best Route ({algo_name})\nDistance: {total_distance(best_path):.3f}")
    plt.legend()
    plt.grid(True)
    
    # Save the best route plot as PNG
    plt.savefig(filename)
    plt.close()

# Best paths
best_hc_index = np.argmin(hc_distances)
best_sa_index = np.argmin(sa_distances)

# Plot and save best routes for both algorithms
plot_best_route(hc_paths[best_hc_index], "Hill Climbing", "blue", "best_route_hc.png")
plot_best_route(sa_paths[best_sa_index], "Simulated Annealing", "green", "best_route_sa.png")

In [20]:
# ------------------------ GIF ANIMATION ------------------------

import matplotlib.animation as animation

def animate_path(path_history, filename, color, title):
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.scatter(cities[:, 0], cities[:, 1], c='black')
    line, = ax.plot([], [], color=color, lw=2)
    title_obj = ax.set_title(title)

    def update(frame):
        path = path_history[frame]
        coords = cities[path + [path[0]]]
        line.set_data(coords[:, 0], coords[:, 1])
        title_obj.set_text(f"Step {frame} | Distance: {total_distance(path):.3f}")
        return line, title_obj

    ani = animation.FuncAnimation(fig, update, frames=len(path_history), interval=200, blit=False)
    ani.save(filename, writer='pillow', fps=5)
    plt.close()

# Save GIF for Hill Climbing
animate_path(hc_paths, "hill_climbing_animation.gif", "blue", "Hill Climbing Path Animation")

# Save GIF for Simulated Annealing
animate_path(sa_paths, "simulated_annealing_animation.gif", "green", "Simulated Annealing Path Animation")

print("Charts and GIFs saved successfully!")


Charts and GIFs saved successfully!
